<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/21-state_dict().ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `state_dict()`
`torch.nn.Module` 객체의 `state_dict()` 메서드.

- 모델의 학습 가능한 parameters의 상태와
- 버퍼(예: BatchNorm의 running mean과 variance 등)의 상태를 저장하는
- collections.OrderedDict 객체를 반환.
- 반환된 객체는 모델의 현재 상태를 나타내며, 저장 및 로드가 가능.

1. OrderedDict 형태:
- attribute 이름을 키로 하고,
- 그에 대응하는 torch.Tensor를 값으로 갖는
- `collections.OrderedDict` 객체를 반환.
2. 학습 가능한 매개변수:
- `torch.nn.Parameter` 객체로 정의된 모든 학습 가능한 attributes를 포함.
3. 버퍼:
- 모델에 포함된 모든 버퍼(예: BatchNorm 계층의 running mean과 variance)도 포함.

- Serialization: 개체를 key: value 형태로 하여 직렬화로 저장.
- OrderedDict: 순서가 정의된 딕셔너리 형태.



## parameters
`state_dict(desitnation=None, prefix='', keep_vars=False)`
### keep_vars
buffers와 parameters의 값만을 추출할지 결정
- 저장의 용도로는 keep_vars=False를 사용하는 게 좋음.

**keep_vars=True인 경우**
- value가 parameter인 경우엔 Parameter 로 얻어짐.
- value가 buffer인 경우엔 Tensor 로 얻어짐.
- 메모리 사용량이 커지고, 매우 느리고 복잡한 동작 발생.

**장점**
- 모델 디버깅: 모델 상태를 조사하고 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
- 모델 커스터마이징: 모델을 불러온 후 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
- 모델 저장 및 불러오기 확장: 모델 저장 및 불러오기 프로세스를 확장하고 추가적인 정보를 저장해야 하는 경우 유용



In [1]:
import torch
import torch.nn as nn
from collections import OrderedDict

In [2]:
# 간단한 모델 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear = nn.Linear(10, 5)
        self.bn = nn.BatchNorm1d(5)

    def forward(self, x):
        x = self.linear(x)
        x = self.bn(x)
        return x

# 모델 인스턴스 생성
model = MyModel()

# 모델의 state_dict 가져오기
state_dict = model.state_dict()

# state_dict 내용 출력
for key, value in state_dict.items():
    print(f"{key}: {value.shape}")

linear.weight: torch.Size([5, 10])
linear.bias: torch.Size([5])
bn.weight: torch.Size([5])
bn.bias: torch.Size([5])
bn.running_mean: torch.Size([5])
bn.running_var: torch.Size([5])
bn.num_batches_tracked: torch.Size([])


### `linear`: `nn.Linear`
- `linear.weight`: weight parameter
- `linear.bias`: bias parameter

### `bn`: `nn.BatchNorm1d`
- `bn.weight`: weight parameter
- `bn.bias`: bias parameter
- `bn.running_mean`: running mean buffer
- `bn.running_var`: running variance buffer
- `bn.num_batches_tracked`: 배치의 수 추적 buffer

In [3]:
# 모델의 state_dict 가져오기
state_dict0 = model.state_dict(prefix='ds', keep_vars=True)

# state_dict 내용 출력
for key, value in state_dict0.items():
    print(f"{key}: {value.shape}")

dslinear.weight: torch.Size([5, 10])
dslinear.bias: torch.Size([5])
dsbn.weight: torch.Size([5])
dsbn.bias: torch.Size([5])
dsbn.running_mean: torch.Size([5])
dsbn.running_var: torch.Size([5])
dsbn.num_batches_tracked: torch.Size([])


## 모델 저장

state_dict를 파일에 저장하여 나중에 모델을 복원할 수 있음.

In [4]:
torch.save(model.state_dict(), 'model_state.pth')

## 모델 로드

저장된 state_dict를 로드하여 모델을 복원.

In [5]:
model = MyModel()
model.load_state_dict(torch.load('model_state.pth'))

<All keys matched successfully>

### 파라미터 업데이트

state_dict를 사용하여 모델의 특정 파라미터를 업데이트할 수 있음.

In [6]:
state_dict['linear.weight'] = torch.ones_like(state_dict['linear.weight'])
model.load_state_dict(state_dict)

<All keys matched successfully>